In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from dask.array.core import normalize_chunks
from pathlib import Path
from sgkit_bgen import read_bgen
from bokeh.io import output_notebook, export_png
from dask.diagnostics import ResourceProfiler, ProgressBar, Profiler, visualize
%load_ext autoreload
%autoreload 2
output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(0,'../scripts')
from convert_genetic_data import BGENPaths, Contig, load_bgen, load_bgen_samples, load_bgen_probabilities, save_dataset

In [3]:
paths = BGENPaths(
    bgen_path='/home/eczech/data/rs-ukb-local/bgen/ukb_imp_chrXY_v3.bgen',
    variants_path='/home/eczech/data/rs-ukb-local/bgen/ukb_mfi_chrXY_v3.txt',
    samples_path='/home/eczech/data/rs-ukb-local/bgen/ukb59384_imp_chrXY_v3_s486331.sample',
)

In [4]:
n_bytes = 536870912 # 512MiB
#n_bytes = 268435456 # 256MiB
n_variants = 1024
n_samples = (n_bytes // 4) // n_variants
n_variants, n_samples

(1024, 65536)

In [8]:
[t[0] for t in normalize_chunks('512MiB', shape=(90_000_000, 486443, 2), dtype='uint8')]

[16000, 16384, 2]

In [10]:
paths = BGENPaths(
    bgen_path='/home/eczech/data/rs-ukb-local/bgen/ukb_imp_chrXY_v3.bgen',
    variants_path='/home/eczech/data/rs-ukb-local/bgen/ukb_mfi_chrXY_v3.txt',
    samples_path='/home/eczech/data/rs-ukb-local/bgen/ukb59384_imp_chrXY_v3_s486331.sample',
)
contig = Contig(index=25, name='XY')
ds = load_bgen(paths, contig, chunks=(n_variants, n_samples, 3))
ds

<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:15...
    variant_rsid                    (variants) object 'rs370048753' ... 'X:15...
    variant_position                (variants) int32 60014 60014 ... 155260523
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'T' 'G'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'G' 'T'
    variant_maf                     (variants) float64 0.0003958 ... 0.4857
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'T' 'T'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.3021
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sample_id2                      (samples) int32 4476413 3205773 ... 4315851
    sample_missing                  (samples) object '0' '0' '0' ... '0' '0' '0'
    sample_sex                      (samples) object '1' '2' '2' ... '1' '2' '1'
    variant_contig                  (variants) int16 25 25 25 25 ... 25 25 25 25
    variant_contig_name             (variants) <U4 'PAR1' 'XY' ... 'XY' 'XY'
    call_genotype_probability       (variants, samples, genotypes) float32 dask.array<chunksize=(10000, 10000, 2), meta=np.ndarray>
    call_genotype_probability_mask  (variants, samples) bool dask.array<chunksize=(10000, 10000), meta=np.ndarray>
Attributes:
    contigs:       ['PAR1', 'XY']
    contig_name:   XY
    contig_index:  25

In [11]:
ds.call_genotype_probability[:3, :3].compute()

<xarray.DataArray 'call_genotype_probability' (variants: 3, samples: 3, genotypes: 2)>
array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]], dtype=float32)
Dimensions without coordinates: variants, samples, genotypes

In [7]:
ds.call_genotype_probability_mask[:3, :3].compute()

<xarray.DataArray 'call_genotype_probability_mask' (variants: 3, samples: 3)>
array([[False, False, False],
       [False, False, False],
       [False, False, False]])
Dimensions without coordinates: variants, samples

### Test Compute

In [7]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    m = ds['call_dosage'][:2500,:].mean(dim='samples').compute(scheduler='processes')

[########################################] | 100% Completed |  2min 18.6s
CPU times: user 43.3 s, sys: 1.11 s, total: 44.4 s
Wall time: 2min 19s


In [9]:
pd.Series(m).describe()

count    2500.000000
mean        0.115621
std         0.320614
min         0.000021
25%         0.000411
50%         0.001762
75%         0.011835
max         1.912868
dtype: float64

### Test Save Locally

In [6]:
dss = ds.sel(variants=np.s_[:2500])
dss

<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:29...
    variant_rsid                    (variants) object 'rs370048753' ... 'rs28...
    variant_position                (variants) int32 60014 60014 ... 293124
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'G' 'A'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_maf                     (variants) float64 0.0003958 ... 0.0008407
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.8284
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sample_id2                      (samples) int32 4476413 3205773 ... 4315851
    sample_missing                  (samples) object '0' '0' '0' ... '0' '0' '0'
    sample_sex                      (samples) object '1' '2' '2' ... '1' '2' '1'
    variant_contig                  (variants) int16 25 25 25 25 ... 25 25 25 25
    variant_contig_name             (variants) <U4 'PAR1' 'XY' ... 'XY' 'XY'
    call_genotype_probability       (variants, samples, genotypes) float32 dask.array<chunksize=(1024, 65536, 2), meta=np.ndarray>
    call_genotype_probability_mask  (variants, samples) bool dask.array<chunksize=(1024, 65536), meta=np.ndarray>
Attributes:
    contigs:       ['PAR1', 'XY']
    contig_name:   XY
    contig_index:  25

In [8]:
%%time
# Export sample WITHOUT GP compression filter (peaking at about 8G of 14G used on 4 workers w/ 256 MiB chunks)
with ProgressBar(), ResourceProfiler() as prof:
    save_dataset('/tmp/test.without_filter.zarr', dss, contig, scheduler='processes', remote=False, rescale_gp=False)

2020-08-20 14:55:11,433 | convert_genetic_data | INFO | Dataset for contig Contig(name='XY', index=25):
<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:29...
    variant_rsid                    (variants) object 'rs370048753' ... 'rs28...
    variant_position                (variants) int32 60014 60014 ... 293124
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'G' 'A'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_maf                     (variants) float64 0.0003958 ... 0.0008407
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.8284
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sam

In [7]:
%%time
# Export sample with GP compression filter (clevel 5)
with ProgressBar(), ResourceProfiler() as rprof, Profiler() as prof:
    save_dataset('/tmp/test.with_filter_clevel5.zarr', dss, contig, scheduler='processes', remote=False, rescale_gp=True)

2020-08-20 21:15:10,190 | convert_genetic_data | INFO | Dataset for contig Contig(name='XY', index=25):
<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:29...
    variant_rsid                    (variants) object 'rs370048753' ... 'rs28...
    variant_position                (variants) int32 60014 60014 ... 293124
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'G' 'A'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_maf                     (variants) float64 0.0003958 ... 0.0008407
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.8284
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sam

In [8]:
visualize([prof, rprof])

Column(id='1126', ...)

In [7]:
%%time
# Export sample with GP compression filter and clevel 7
with ProgressBar(), ResourceProfiler() as rprof, Profiler() as prof:
    save_dataset('/tmp/test.with_filter_clevel7.zarr', dss, contig, scheduler='processes', remote=False, rescale_gp=True)

2020-08-20 22:34:31,946 | convert_genetic_data | INFO | Dataset for contig Contig(name='XY', index=25):
<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:29...
    variant_rsid                    (variants) object 'rs370048753' ... 'rs28...
    variant_position                (variants) int32 60014 60014 ... 293124
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'G' 'A'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_maf                     (variants) float64 0.0003958 ... 0.0008407
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.8284
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sam

In [8]:
visualize([prof, rprof])

Column(id='1126', ...)

In [7]:
%%time
# Export sample with GP compression filter and high clevel (9 instead of 5)
with ProgressBar(), ResourceProfiler() as rprof, Profiler() as prof:
    save_dataset('/tmp/test.with_filter_clevel9.zarr', dss, contig, scheduler='processes', remote=False, rescale_gp=True)

2020-08-20 20:40:54,634 | convert_genetic_data | INFO | Dataset for contig Contig(name='XY', index=25):
<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    variant_id                      (variants) object 'X:60014_T_C' ... 'X:29...
    variant_rsid                    (variants) object 'rs370048753' ... 'rs28...
    variant_position                (variants) int32 60014 60014 ... 293124
    variant_allele1_ref             (variants) object 'T' 'T' 'C' ... 'G' 'A'
    variant_allele2_alt             (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_maf                     (variants) float64 0.0003958 ... 0.0008407
    variant_minor_allele            (variants) object 'C' 'G' 'T' ... 'A' 'G'
    variant_info                    (variants) float64 0.7276 0.6867 ... 0.8284
    sample_id1                      (samples) int32 4476413 3205773 ... 4315851
    sam

In [11]:
visualize([prof, rprof])

Column(id='1366', ...)

In [10]:
# TODO: compare to test.zarr with dosage
! du -ch /tmp/test{,.with_filter_clevel5,.with_filter_clevel7,.with_filter_clevel9,.without_filter}.zarr --max-depth=1

20K	/tmp/test.zarr/variant_position
32K	/tmp/test.zarr/variant_info
16K	/tmp/test.zarr/variant_allele1_ref
16K	/tmp/test.zarr/variant_contig_name
16K	/tmp/test.zarr/variant_minor_allele
24K	/tmp/test.zarr/variant_id
1.5M	/tmp/test.zarr/sample_id1
28K	/tmp/test.zarr/variant_rsid
204K	/tmp/test.zarr/sample_sex
349M	/tmp/test.zarr/call_dosage
44K	/tmp/test.zarr/sample_missing
32K	/tmp/test.zarr/variant_maf
1.5M	/tmp/test.zarr/sample_id2
16K	/tmp/test.zarr/variant_allele2_alt
396K	/tmp/test.zarr/call_dosage_mask
16K	/tmp/test.zarr/variant_contig
352M	/tmp/test.zarr
20K	/tmp/test.with_filter_clevel5.zarr/variant_position
32K	/tmp/test.with_filter_clevel5.zarr/variant_info
16K	/tmp/test.with_filter_clevel5.zarr/variant_allele1_ref
16K	/tmp/test.with_filter_clevel5.zarr/variant_contig_name
16K	/tmp/test.with_filter_clevel5.zarr/variant_minor_allele
24K	/tmp/test.with_filter_clevel5.zarr/variant_id
1.6M	/tmp/test.with_filter_clevel5.zarr/sample_id1
28K	/tmp/test.with_filter_clevel5.zarr/varian

In [26]:
dsr = xr.open_zarr('/tmp/test.with_filter2.zarr', mask_and_scale=False, decode_cf=False)
dsr

<xarray.Dataset>
Dimensions:                         (genotypes: 2, samples: 486443, variants: 2500)
Dimensions without coordinates: genotypes, samples, variants
Data variables:
    call_genotype_probability       (variants, samples, genotypes) uint8 dask.array<chunksize=(1024, 65536, 2), meta=np.ndarray>
    call_genotype_probability_mask  (variants, samples) bool dask.array<chunksize=(1024, 65536), meta=np.ndarray>
    sample_id1                      (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_id2                      (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_missing                  (samples) object dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) object dask.array<chunksize=(60806,), meta=np.ndarray>
    variant_allele1_ref             (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_allele2_alt             (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_contig_name             (variants) <U4 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_id                      (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_minor_allele            (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_rsid                    (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
Attributes:
    contig_index:  25
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [27]:
dsr.call_genotype_probability[:3, :3].compute()

<xarray.DataArray 'call_genotype_probability' (variants: 3, samples: 3, genotypes: 2)>
array([[[2, 2],
        [2, 2],
        [2, 2]],

       [[2, 2],
        [2, 2],
        [2, 2]],

       [[2, 2],
        [2, 2],
        [2, 2]]], dtype=uint8)
Dimensions without coordinates: variants, samples, genotypes
Attributes:
    add_offset:    1
    scale_factor:  0.003937007874015748
    _FillValue:    0

In [15]:
dsr.call_genotype_probability[1:3, :].mean(dim='samples').compute()

<xarray.DataArray 'call_genotype_probability' (variants: 2, genotypes: 2)>
array([[2.19615453, 2.00168159],
       [2.01340753, 2.        ]])
Dimensions without coordinates: variants, genotypes

In [18]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    m = dsr['call_dosage'][:2500,:].mean(dim='samples').compute(scheduler='processes')

[########################################] | 100% Completed |  9.0s
CPU times: user 232 ms, sys: 146 ms, total: 377 ms
Wall time: 10.1 s


In [20]:
pd.Series(m).describe()

count    2500.000000
mean        0.115621
std         0.320614
min         0.000021
25%         0.000411
50%         0.001762
75%         0.011835
max         1.912868
dtype: float64

### Test Save Remotely

In [25]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    save_dataset('gs://rs-ukb/prep-data/test.zarr', dss, contig, scheduler='processes', remote=True)

2020-08-19 00:02:10,341 | convert | INFO | Dataset for contig Contig(name='XY', index=25):
<xarray.Dataset>
Dimensions:               (samples: 486443, variants: 2500)
Dimensions without coordinates: samples, variants
Data variables:
    variant_id            (variants) object 'X:60014_T_C' ... 'X:293124_A_G'
    variant_rsid          (variants) object 'rs370048753' ... 'rs28621836'
    variant_position      (variants) int32 60014 60014 60017 ... 293106 293124
    variant_allele1_ref   (variants) object 'T' 'T' 'C' 'G' ... 'G' 'C' 'G' 'A'
    variant_allele2_alt   (variants) object 'C' 'G' 'T' 'C' ... 'C' 'T' 'A' 'G'
    variant_maf           (variants) float64 0.0003958 0.0005032 ... 0.0008407
    variant_minor_allele  (variants) object 'C' 'G' 'T' 'C' ... 'C' 'T' 'A' 'G'
    variant_info          (variants) float64 0.7276 0.6867 ... 0.3437 0.8284
    sample_id1            (samples) int32 4476413 3205773 ... 2850971 4315851
    sample_id2            (samples) int32 4476413 3205773 ...

In [26]:
import gcsfs
gcs = gcsfs.GCSFileSystem()
store = gcsfs.GCSMap('gs://rs-ukb/prep-data/test.zarr', gcs=gcs, check=False, create=False)
dsr = xr.open_zarr(store)
dsr

<xarray.Dataset>
Dimensions:               (samples: 486443, variants: 2500)
Dimensions without coordinates: samples, variants
Data variables:
    call_dosage           (variants, samples) float32 dask.array<chunksize=(1024, 131072), meta=np.ndarray>
    call_dosage_mask      (variants, samples) bool dask.array<chunksize=(1024, 131072), meta=np.ndarray>
    sample_id1            (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_id2            (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_missing        (samples) object dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex            (samples) object dask.array<chunksize=(60806,), meta=np.ndarray>
    variant_allele1_ref   (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_allele2_alt   (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_contig        (variants) int16 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_contig_name   (variants) <U4 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_info          (variants) float64 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_maf           (variants) float64 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_minor_allele  (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(2500,), meta=np.ndarray>
    variant_rsid          (variants) object dask.array<chunksize=(2500,), meta=np.ndarray>

In [27]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    m = dsr['call_dosage'][:2500,:].mean(dim='samples').compute(scheduler='processes')

[########################################] | 100% Completed | 10.1s
CPU times: user 220 ms, sys: 142 ms, total: 362 ms
Wall time: 11.1 s


In [28]:
pd.Series(m).describe()

count    2500.000000
mean        0.115621
std         0.320614
min         0.000021
25%         0.000411
50%         0.001762
75%         0.011835
max         1.912868
dtype: float64